In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier, StackingRegressor, StackingClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
from sklearn.neighbors import KNeighborsRegressor


#Libraries for visualizing trees
from sklearn.tree import export_graphviz 
from six import StringIO
from IPython.display import Image  
import pydotplus

# Linear Regression Libraries
import statsmodels.formula.api as smf
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# MARS
from pyearth import Earth
import time as time

import xgboost as xgb
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier


np.warnings.filterwarnings('ignore')

In [3]:
# original data
data = pd.read_csv('train.csv')
validation_data = pd.read_csv('test.csv')

In [4]:
# data cleaning
state_freqs = data.addr_state.value_counts().to_frame().reset_index().rename(columns={'index' : "state", "addr_state" : "count"})
other_state = state_freqs[state_freqs['count'] < data.shape[0] * .01]['state'].tolist()

def clean(data):
    
    df = data.copy()
    # addr_state: 'other' category
    for obs in range(df.shape[0]):
        if df.loc[obs, 'addr_state'] in other_state:
            df.loc[obs, 'addr_state'] = 'Other'
            
    # earliest_cr_line: split month, year
    df['earliest_cr_line_month'] = df.earliest_cr_line.str.split('-', expand = True)[0]
    df['earliest_cr_line_year'] = df.earliest_cr_line.str.split('-', expand = True)[1].astype(int)
    df.drop(columns = ['earliest_cr_line'], inplace = True)
    
    # last_credit_pull_d: split month, year
    df['last_credit_pull_d_month'] = df.last_credit_pull_d.str.split('-', expand = True)[0]
    df['last_credit_pull_d_year'] = df.last_credit_pull_d.str.split('-', expand = True)[1].astype(int)
    df.drop(columns = 'last_credit_pull_d', inplace = True)
    
    # sub_grade: remove numeric ratings, group F and G into 'other'
    df['sub_grade_letter'] = df['sub_grade'].str.extract('([A-Z]+)')
    df.loc[df["sub_grade_letter"] == "F", "sub_grade_letter"] = "Other"
    df.loc[df["sub_grade_letter"] == "G", "sub_grade_letter"] = "Other"
    df.drop(columns = 'sub_grade', inplace = True)
    
    # drop 'id' columns
    df.drop(columns = 'id', inplace = True)
    
    return df
    


In [5]:
train = clean(data) # train 
test = clean(validation_data) # kaggle 

In [6]:
# filter for predictors
X = train.drop(columns = ['money_made_inv'])
# filter for response
y = train['money_made_inv']

# 70% of training data becomes the new train set, 30% becomes new test set
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size = int(train.shape[0]*.7), random_state = 1)


In [7]:
# numeric predictors
Xtrain_num = Xtrain.select_dtypes(include=np.number)
Xtest_num = Xtest.select_dtypes(include=np.number)

# categorical predictors
Xtrain_cat = Xtrain[['sub_grade_letter', 'term', 'initial_list_status', 'application_type']]
Xtest_cat = Xtest[['sub_grade_letter', 'term', 'initial_list_status', 'application_type']]

# numeric and categorical predictors
Xtrain_num_cat = pd.concat([Xtrain_num, pd.get_dummies(Xtrain_cat)],axis=1)
Xtest_num_cat = pd.concat([Xtest_num, pd.get_dummies(Xtest_cat)],axis=1)


# AdaBoost

In [11]:
model = AdaBoostRegressor(random_state=1)
grid = dict()
grid['n_estimators'] = [100, 200, 500]
grid['learning_rate'] = [0.0001, 0.001, 0.01,0.1, 1.0]
grid['base_estimator'] = [DecisionTreeRegressor(max_depth=5), 
                          DecisionTreeRegressor(max_depth=10),
                          DecisionTreeRegressor(max_depth=15)]


# define the evaluation procedure
cv = KFold(n_splits=5, shuffle=True, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, 
                           cv=cv, scoring='neg_mean_squared_error')


# execute the grid search
grid_result = grid_search.fit(Xtrain_num_cat, ytrain)


# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']


#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param)

Best: -668132.332214 using {'base_estimator': DecisionTreeRegressor(max_depth=15), 'learning_rate': 1.0, 'n_estimators': 500}


In [14]:
# choose optimal params: learning_rate = .01, n_estimators = 500, DecisionTreeRegressor(max_depth=10)
cv_results = pd.DataFrame(grid_result.cv_results_)
cv_results.sort_values(by = 'mean_test_score', ascending = False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_base_estimator,param_learning_rate,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
44,35.284385,1.409320,0.404194,0.016356,DecisionTreeRegressor(max_depth=15),1.0,500,{'base_estimator': DecisionTreeRegressor(max_d...,-5.227475e+05,-3.369004e+05,-7.006135e+05,-7.922475e+05,-9.881528e+05,-6.681323e+05,223483.340215,1
42,9.984785,0.654362,0.118767,0.014342,DecisionTreeRegressor(max_depth=15),1.0,100,{'base_estimator': DecisionTreeRegressor(max_d...,-5.476090e+05,-3.240299e+05,-7.377835e+05,-8.190899e+05,-9.747393e+05,-6.806503e+05,225298.076586,2
43,18.527259,0.284580,0.216166,0.027287,DecisionTreeRegressor(max_depth=15),1.0,200,{'base_estimator': DecisionTreeRegressor(max_d...,-5.332480e+05,-3.461538e+05,-7.223528e+05,-8.045662e+05,-1.002178e+06,-6.816998e+05,225425.902902,3
29,42.134973,0.183349,0.606737,0.060186,DecisionTreeRegressor(max_depth=10),1.0,500,{'base_estimator': DecisionTreeRegressor(max_d...,-5.334770e+05,-3.569231e+05,-6.913834e+05,-7.870247e+05,-1.046737e+06,-6.831091e+05,233186.285698,4
28,17.519178,0.523208,0.205984,0.021753,DecisionTreeRegressor(max_depth=10),1.0,200,{'base_estimator': DecisionTreeRegressor(max_d...,-5.336235e+05,-3.526723e+05,-6.751921e+05,-8.088928e+05,-1.047892e+06,-6.836545e+05,236847.510390,5
25,22.323429,1.346329,0.273777,0.078706,DecisionTreeRegressor(max_depth=10),0.1,200,{'base_estimator': DecisionTreeRegressor(max_d...,-5.464934e+05,-4.225440e+05,-7.216688e+05,-7.850186e+05,-9.509020e+05,-6.853254e+05,184455.364434,6
38,72.967605,0.141594,0.486264,0.024437,DecisionTreeRegressor(max_depth=15),0.01,500,{'base_estimator': DecisionTreeRegressor(max_d...,-5.243430e+05,-4.225543e+05,-6.624871e+05,-9.048094e+05,-9.478603e+05,-6.924108e+05,206073.931429,7
26,52.207444,0.443498,0.598040,0.050157,DecisionTreeRegressor(max_depth=10),0.1,500,{'base_estimator': DecisionTreeRegressor(max_d...,-5.594572e+05,-4.421744e+05,-7.244623e+05,-7.341141e+05,-1.011281e+06,-6.942977e+05,192209.053361,8
24,10.928781,0.281483,0.097102,0.007662,DecisionTreeRegressor(max_depth=10),0.1,100,{'base_estimator': DecisionTreeRegressor(max_d...,-5.583312e+05,-4.398466e+05,-6.974702e+05,-8.904955e+05,-9.304057e+05,-7.033098e+05,188189.686995,9
27,9.105467,0.948967,0.095963,0.008856,DecisionTreeRegressor(max_depth=10),1.0,100,{'base_estimator': DecisionTreeRegressor(max_d...,-5.194410e+05,-3.495966e+05,-7.692441e+05,-8.635592e+05,-1.024845e+06,-7.053372e+05,241790.209089,10


In [15]:
model_ada = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=10),
                              n_estimators=500,
                              learning_rate=.01,
                              random_state=1).fit(Xtrain_num_cat, ytrain)

In [16]:
pred_ada = model_ada.predict(Xtest_num_cat)
np.sqrt(mean_squared_error(pred_ada,ytest))


895.938134651224

In [18]:
model_ada = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=15),
                              n_estimators=200,
                              learning_rate=.0001,
                              random_state=1).fit(Xtrain_num_cat, ytrain)


In [19]:
pred_ada = model_ada.predict(Xtest_num_cat)
np.sqrt(mean_squared_error(pred_ada,ytest))


933.1985968718362

In [20]:
model_ada = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=15),
                              n_estimators=500,
                              learning_rate=1,
                              random_state=1).fit(Xtrain_num_cat, ytrain)



In [21]:
pred_ada = model_ada.predict(Xtest_num_cat)
np.sqrt(mean_squared_error(pred_ada,ytest))


871.623204011823